In [ ]:
def search_body(query, N=100):
   """ 
    Gets the best N results for a given query using TFIDF and cosine similarity 
    
    Parameters:
    -----------
    query: String. The query.

    N: Integer. How many documents to retrieve. By default N = 100.
    
    Returns:
    -----------
    return: a list of up to N search results, ordered from best to worst where
     each element is a tuple (wiki_id, title).
    """
  pass

In [ ]:
def search_title(query):
   """ 
    Gets all the results for a given query using binary ranking on the titles 
    
    Parameters:
    -----------
    query: String. The query.
    
    Returns:
    -----------
    return: a list of all the search results, ordered from best to worst where
     each element is a tuple (wiki_id, title).
    """
  pass

In [ ]:
def search_anchor(query):
    """ 
    Gets all the results for a given query using binary ranking on the anchor texts 
    
    Parameters:
    -----------
    query: String. The query.
    
    Returns:
    -----------
    return: a list of all the search results, ordered from best to worst where
     each element is a tuple (wiki_id, title).
    """
  pass

In [ ]:
def get_pagerank(article_ids):
    """ 
    Gets the number of page views that each of the provide wiki articles
        had in August 2021.
    
    Parameters:
    -----------
    article_ids: List. list of article ids of which to return the page views of.
    
    Returns:
    -----------
    return: a list of ints:
          list of page view numbers from August 2021 that correrspond to the 
          provided list article IDs.
    """
  pass

In [ ]:
def get_pageview(article_ids):
    """ 
    Gets PageRank values for a list of provided wiki article IDs.
    
    Parameters:
    -----------
    article_ids: List. list of article ids of which to return the page ranks of.
    
    Returns:
    -----------
    return: a list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    """
  pass

In [ ]:
def search(query):
  pass